In [68]:
year = 2025; day = 9

from aocd import get_data, submit
import numpy as np
data = get_data(year=year, day=day)

# data = """7,1
# 11,1
# 11,7
# 9,7
# 9,5
# 2,5
# 2,3
# 7,3"""
data=data.split("\n")
data = [list(map(int, line.split(","))) for line in data]
data = np.array(data)

from helpers.helpers import get_rel_coords, directions_map
from itertools import combinations

In [69]:
len(data)**2

246016

In [70]:
%pip install tqdm


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [71]:
W, H = (max(data[:,1])+2, max(data[:,0]+2))
W, H

(np.int64(98343), np.int64(98464))

In [72]:
from tqdm import tqdm
max_surf = 0
for a, b in tqdm(combinations(data, 2)):
    surf = (abs(a[0] - b[0]) + 1) * (abs(a[1] - b[1]) + 1)
    if surf > max_surf:
        max_surf = surf
max_surf = int(max_surf)
max_surf

122760it [00:00, 1125547.06it/s]


4758121828

In [73]:
len(list(combinations(data, 2)))

122760

In [74]:
x, y = data[-1]
edges = set()
# edges.add((x,y))
# find interior 
for nx, ny in data:
    if x == nx: 
        sy = min(y, ny)
        for dy in range(sy, max(y, ny)+1):
            edges.add((int(x), dy))
    if y == ny:
        sx = min(x, nx)
        for dx in range(sx, max(x, nx)+1):
            edges.add((dx, int(y)))
    x, y = nx, ny

In [75]:
# def ray_cast(end_x, y):
#     if (end_x, y) in edges: 
#         return True
#     transition_count = 0
#     prev = False
#     for x in range(end_x):
#         if (x, y) not in edges and prev:
#             transition_count += 1
#         prev = (x, y) in edges
#     return transition_count % 2

In [78]:
unique_xs = set(np.unique(data[:, 0]))
padded_unique_xs = set()
for x in unique_xs:
    padded_unique_xs.add(x-1)
    padded_unique_xs.add(x)
    padded_unique_xs.add(x+1)
padded_unique_xs = list(padded_unique_xs)

len(padded_unique_xs)

744

In [79]:
def ray_cast(end_x, y):
    if (end_x, y) in edges:
        return True
    transition_count = 0
    prev_dir = None
    # for x in range(end_x+1):
    for x in [0]+padded_unique_xs:
        if x > end_x:
            break
        if (x, y) in edges:
            if (x, y-1) in edges and (x, y+1) in edges:
                transition_count += 1
            elif (x, y-1) in edges:
                if prev_dir == "down":
                    transition_count += 1
                prev_dir = "up"
            elif (x, y+1) in edges:
                if prev_dir == "up":
                    transition_count += 1
                prev_dir = "down"
            else: 
                pass
        else: 
            prev_dir = None
        # prev = (x, y) in edges
    return transition_count % 2 == 1

In [121]:
from itertools import combinations
# square is valid if 
# - no other points are in the interior
# - corners are not outside of the interior

max_surf = 0
# for a, b in tqdm(combinations(data, 2)):
a = (94865, 50110)
# a = (94865, 48656)
for b in data:
    # if b[1] > 48656:
        # continue
    x1, x2 = min(a[0], b[0]), max(a[0], b[0])
    y1, y2 = min(a[1], b[1]), max(a[1], b[1])
    # opposite corners should be in interior or in edges
    # if not ray_cast(a[0], b[1]):
    #     continue
    # if not ray_cast(b[0], a[1]):
    #     continue
    # no other points should be inside our rectangle
    interior_mask = (x1 < data[:, 0]) & (data[:, 0] < x2) & (y1 < data[:, 1]) & (data[:, 1] < y2)
    if np.any(interior_mask):
        continue
    
    surf = (abs(a[0] - b[0]) + 1) * (abs(a[1] - b[1]) + 1)
    if surf > max_surf:
        print(a, b, surf)

        max_surf = surf
max_surf

(94865, 50110) [97997 50117] 25064
(94865, 50110) [97997 51346] 3875521
(94865, 50110) [97943 52549] 7512760
(94865, 50110) [97943 53788] 11327641
(94865, 50110) [97502 54948] 12765282
(94865, 50110) [97502 56154] 15946710
(94865, 50110) [97395 57433] 18537044
(94865, 50110) [97292 58596] 20606436
(94865, 50110) [97292 59837] 23619584
(94865, 50110) [97281 61017] 26364636
(94865, 50110) [ 5698 67727] 1570961824
(94865, 50110) [ 5301 67727] 1577956170


np.int64(1577956170)

In [ ]:
233683680
4758121828

In [173]:
interiors = set()
for x in range(20):
    for y in range(20):
        if ray_cast(x, y):
            interiors.add((x, y))



In [85]:
%pip install matplotlib


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
[d for d in data if d[0] == 94865]

[array([94865, 50110]), array([94865, 48656])]

In [102]:
# ray_cast(94865, 50110)
ray_cast(2171, 50110)

False

In [124]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot all line segments connecting consecutive points
for i in range(len(data)):
    a = data[i]
    b = data[(i + 1) % len(data)]  # Connect last point back to first
    fig.add_trace(go.Scatter(
        x=[a[0], b[0]], 
        y=[a[1], b[1]], 
        mode='lines',
        line=dict(color='blue', width=1),
        showlegend=False,
        hoverinfo='skip'
    ))

# Plot the points
fig.add_trace(go.Scatter(
    x=data[:, 0], 
    y=data[:, 1], 
    mode='markers',
    marker=dict(color='red', size=5),
    name='Points',
    hovertemplate='(%{x}, %{y})<extra></extra>'
))

# Add rectangle

# x1, y1 = 2172, 50110
# x2, y2 = 94865, 48656

# x1, y1 = 5301, 67727
# x2, y2 = 94865, 50110

x1, y1 = 5301, 67727
x2, y2 = 94865, 50110

# x1, y1 = 4270, 35611
# x2, y2 = 94865, 48656

# x1, y1 = 96089, 62054
# x2, y2 = 3347, 63224
x1, y1 = 14813, 83283
x2, y2 = 84780, 17100
rect_x = [min(x1, x2), max(x1, x2), max(x1, x2), min(x1, x2), min(x1, x2)]
rect_y = [min(y1, y2), min(y1, y2), max(y1, y2), max(y1, y2), min(y1, y2)]
fig.add_trace(go.Scatter(
    x=rect_x, 
    y=rect_y, 
    mode='lines',
    line=dict(color='green', width=2),
    name='Rectangle',
    hoverinfo='skip'
))

fig.update_layout(
    xaxis_title='X',
    yaxis_title='Y',
    showlegend=True,
    # width=600,
    height=600,
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

fig.show()

In [125]:
len(edges)

593246

In [ ]:
94664, 34550
4270, 35611

In [112]:
print(f"{x1=} {y1=} {x2=} {y2=} {abs((x1-x2)*(y1-y2))}")

x1=4270 y1=35611 x2=94865 y2=48656 1181811775


In [ ]:
# wrong

# x1=5301 y1=67727 x2=94865 y2=50110 1577848988

In [ ]:
1577848988
1181811775

In [109]:
(x1-x2)*(y1-y2)

-1577848988

In [ ]:
67976
5301, 67727

In [ ]:
6736, 69749

In [176]:
import plotly.express as px
# px.imshow(edges
# )

plot_data = np.zeros((max(data[:,1])+3, max(data[:,0]+3)))
for x, y in all_visited:
    plot_data[y, x] = 0.5
for x, y in edges:
    plot_data[y, x] = 1
for x, y in interiors:
    plot_data[y, x] = 0.3

px.imshow(plot_data)

In [ ]:
#b 
# right
# 1577956170

# wrong
# 134869770

# too low 
# 2396672

# 4630762112
# wrong 
# 4757983825

In [27]:

def simple_bfs(initial_state, m):
    all_visited = set()
    to_visit = [initial_state]
    for iteration in range(1_000):
        new_to_visit = set()
        print(len(to_visit))
        for s in to_visit:
            for new_s in (...):
                if some_condition:
                    new_to_visit.add(new_s)
                    all_visited.add(new_s)
        if not new_to_visit:
            print("DONE")
            break
        to_visit = list(new_to_visit)

    else:
        raise ValueError("max iterations exceeded")